In [7]:
import base64
import json
import re
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Encode the image as base64
with open("user_upload.jpg", "rb") as f:
    img_base64 = base64.b64encode(f.read()).decode("utf-8")

# Create chat messages
messages = [
    {
        "role": "system",
        "content": (
            "You are a meme assistant. Generate a short, funny meme story from the given image. "
            "Return captions, tags, character, and scam_type in valid JSON format ONLY. "
            "Do NOT include any explanations, extra text, or emojis. "
            "The JSON must have keys: caption, tags, character, scam_type. "
            "If scam_type is not applicable, set it to 'Unknown'."
        )
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Generate a meme story for this image."
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{img_base64}"
                }
            }
        ]
    }
]

# Call GPT
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    temperature=0.7
)

# Extract raw output
raw_output = response.choices[0].message.content

# Optional: Extract JSON if GPT wraps it in extra text
match = re.search(r"\{.*\}", raw_output, re.DOTALL)
json_text = match.group() if match else "{}"

# Parse JSON safely
try:
    result = json.loads(json_text)
except json.JSONDecodeError:
    result = {"caption": raw_output, "tags": [], "character": "", "scam_type": "Unknown"}

# Pretty-print result
print(json.dumps(result, indent=4))

llm = ChatOpenAI(model="gpt-4o", temperature=0.7)

prompt = (
    "You are a scriptwriter for an 8-second meme video.\n\n"
    f"Here’s the meme data:\n{result}\n\n"
    "Write a short script for an 8-second video based on this meme. "
    "Make it funny, clear, and with simple dialogue. "
    "Return ONLY the script text, no JSON, no metadata."
)

response = llm.invoke(prompt)

script_text = response.content.strip()
print("🎬 Generated Script:\n", script_text)



{
    "caption": "When you realize that 'unusual activity' is just your cat stepping on your phone.",
    "tags": [
        "scam",
        "funny",
        "cat",
        "textmessage"
    ],
    "character": "Cat",
    "scam_type": "Phishing"
}
🎬 Generated Script:
 [Scene: A close-up of a phone screen with a notification: "Unusual activity detected."]

[Cut to: A cat sitting on the phone, looking innocent.]

Text Overlay: "Security Alert! 🚨"

Cat (voiceover): "I just wanted to order more treats..."

[Cut to: Person picking up the phone, shaking their head and smiling.]

Person (laughing): "Busted, Mr. Whiskers!"

[End scene with the cat giving a playful meow.]


In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from elevenlabs.client import ElevenLabs
from elevenlabs.play import play
import os

load_dotenv()

# LLM setup
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# Narration bot prompt
prompt_template = ChatPromptTemplate.from_messages([
    ("system", 
     "You are a narration assistant. Rewrite scripts into smooth, human-like narration "
     "for a short 8-second video. Keep it conversational and easy for TTS. "
     "Do NOT include stage directions, emojis, or formatting. Just plain narration text."),
    ("user", "Convert this script into narration:\n\n{script}")
])

# Format and run
messages = prompt_template.format_messages(script=script_text)
response = llm.invoke(messages)

narration = response.content.strip()

print("🎤 Narration Text for 11Labs:\n", narration)

elevenlabs = ElevenLabs(
  api_key=os.getenv("ELEVENLABS_API_KEY"),
)

audio = elevenlabs.text_to_speech.convert(
    text= narration ,
    voice_id="JBFqnCBsd6RMkjVDRZzb",
    model_id="eleven_multilingual_v2",
    output_format="mp3_44100_128",
)

play(audio)

🎤 Narration Text for 11Labs:
 A notification pops up on the phone saying, "Unusual activity detected." Meanwhile, a cat sits on the phone, looking completely innocent. The cat thinks, "I just wanted to order more treats." Then, a person picks up the phone, shakes their head, and laughs, saying, "Busted, Mr. Whiskers!" The scene wraps up with the cat giving a playful meow.


In [ ]:
from runwayml import RunwayML

# The env var RUNWAYML_API_SECRET is expected to contain your API key.
client = RunwayML()

task = client.image_to_video.create(
  model='gen4_turbo',
  prompt_image='https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTZouypNVCAMcyEZHJ_I1l1kiyut3KdrVvkTg&s',
  prompt_text='The bunny is eating a carrot',
  ratio='1280:720',
).wait_for_task_output()

print(task)

PermissionDeniedError: Error code: 403 - {'error': 'Model variant gen4_image is not available', 'docUrl': 'https://docs.dev.runwayml.com/api'}

In [55]:
text_entry = f"{result['caption']} | Tags: {', '.join(result['tags'])} | Character: {result['character']} | Scam Type: {result['scam_type']}"
print(text_entry)

When you get a text from your 'bank' but you haven't even opened an account yet. | Tags: scam, phishing, funny, textmessage | Character: Confused User | Scam Type: Phishing


In [49]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vector = embeddings.embed_query(text_entry) 

In [53]:
from langchain.vectorstores import Chroma

# Initialize or load DB
db = Chroma(persist_directory="meme_db", embedding_function=embeddings)

# Add the new meme
db.add_texts([text_entry])
db.persist()

print("✅ Meme stored in vector DB")

✅ Meme stored in vector DB


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
import json

# Serialize for prompt
meme_text = json.dumps(result, indent=4)

# Define prompt template
prompt_template = ChatPromptTemplate.from_messages([
    {
        "role": "system",
        "content": (
            "You are a script writer for 8-second educational meme videos. "
            "Given the meme JSON, generate a short video script describing the scene, "
            "character actions, and any dialogue. "
            "Return output in JSON with keys: 'script', 'duration_seconds'."
        )
    },
    {
        "role": "user",
        "content": f"Generate a video script from this meme JSON:\n{meme_text}"
    }
])

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# Generate script
response = llm(prompt_template.format_prompt().to_messages())

# Extract output
raw_output = response[0].content

# Parse JSON safely
try:
    script_result = json.loads(raw_output)
except json.JSONDecodeError:
    script_result = {"script": raw_output, "duration_seconds": 8}

# Print generated video script
print(json.dumps(script_result, indent=4))


In [ ]:
new_caption = "A dog holding a pizza slice like a phone"
db.add_texts([new_caption])


In [ ]:
query = "A person holding a bag of free money"
results = db.similarity_search(query, k=3)
for r in results:
    print(r.page_content)


In [ ]:
query = "A person holding a bag of free money"
results = db.similarity_search(query, k=3)
for r in results:
    print(r.page_content)
